In [1]:
#compute the uDFT by noting DFT.DFT^* = D, a diagonal matrix, and factoring as D = RR^*, so uDFT = R^{-1}.DFT
#alternately, one could use unitary representations and normalization factors \sqrt{d_\rho/|G|}
#the resulting DFT will have signs \pm 1 on the diagonal DFT.DFT^* = S, which can be factored as s = rr^*, so S=RR^*
#again, uDFT = R^{-1}.DFT

In [2]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z ** ((k//(q+1))%(q-1))

In [3]:
#compute the uDFT by noting DFT.DFT^* = D, a diagonal matrix, and factoring as D = RR^*, so uDFT = R^{-1}.DFT
def unitary_dft():
    dft_matrix = SGA.dft()
    sign_diag = (dft_matrix*dft_matrix.H).diagonal()
    factor_diag_inv = diagonal_matrix([~conj_square_root(d) for d in sign_diag])
    return factor_diag_inv*dft_matrix

In [92]:
#parameters and define the symmetric group algebra
n = 3; q = 5
F = GF(q**2)
G = SymmetricGroup(n)
SGA = G.algebra(F)
assert F.is_finite()
assert F.order().is_square()
if F.characteristic().divides(G.cardinality()):
    raise NotImplementedError("Not implemented when p|n!. Dimension of invariant forms may be greater than one and \sqrt{d_\rho/|G|} is not defined. See modular DFT.")

In [93]:
print(unitary_dft()*unitary_dft().H)

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]


In [94]:
#converting the unitary DFT over finite fields to a complex matrix using the root of unity map
U = unitary_dft(); 

In [95]:
z = F.multiplicative_generator()
log_U = matrix([[U[(i,j)].log(z) if U[(i,j)] != 0 else -1 for j in range(U.nrows())] for i in range(U.nrows())])

In [96]:
print(log_U)

[ 0  0  0  0  0  0]
[21  3  3 15 15  9]
[-1 18  6  6 18 -1]
[-1 22 10 22 10 -1]
[21  3  3  3  3 21]
[ 0  0  0 12 12 12]


In [97]:
to_root_of_unity = lambda a: 0 if a ==0 else exp(2*pi*I*a.log(F.multiplicative_generator())/(F.order()-1))

In [98]:
U_complex = matrix(CC,[[to_root_of_unity(U[(i,j)]) for j in range(U.nrows())] for i in range(U.nrows())])

In [99]:
gram = U_complex*U_complex.H

In [100]:
def round_complex(z, digits):
    if z.imag_part():
        return round(z.real_part(), digits) + round(z.imag_part(), digits) * I
    return round(z, digits)

In [101]:
gram_rounded = gram.apply_map(lambda u:round_complex(u,2))

In [102]:
print(gram_rounded)

[          6.0           0.0           0.0           0.0 4.24 - 1.41*I           0.0]
[          0.0           6.0           0.0           0.0           0.0 4.24 + 1.41*I]
[          0.0           0.0           4.0           0.0           0.0           0.0]
[          0.0           0.0           0.0           4.0           0.0           0.0]
[4.24 + 1.41*I           0.0           0.0           0.0           6.0           0.0]
[          0.0 4.24 - 1.41*I           0.0           0.0           0.0           6.0]
